# Bundesliga YOLOv5 specialized for ball detection


This notebook referred to the following Mr.Shinmura's notebook and dataset.
* https://www.kaggle.com/code/shinmurashinmura/dfl-yolov5-ball-detection#YOLOv5-ball-detection
* https://www.kaggle.com/datasets/shinmurashinmura/dfl-yolov5l6-ball-detection


#### YOLOv5
* https://github.com/ultralytics/yolov5

### Specially trained weight optimized for the balls seen in game movies
The generally provided weight of YOLOv5 includes a ball (sports ball) in its class, but it is difficult to detect the balls in the game movies. In order to detect it, "specially trained weight optimized for the game movies" is required. Thankfully Mr.Shinmura created it and provided in a public dataset. 

In [ ]:
#!rm -rf

In [ ]:
#!pip install moviepy -q

In [ ]:
# Download YOLOv5
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
# Install dependencies
%pip install -qr requirements.txt  # install dependencies

import torch
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

In [ ]:
import os
import gc
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
import shutil
from shutil import copyfile
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import subprocess

# Set weight(pt file) and mp4 file

In [ ]:
shutil.copy('/kaggle/input/dfl-yolov5l6-ball-detection/yolov5l6_trained_600images.pt','/kaggle/working/yolov5')
shutil.copy('/kaggle/input/dfl-bundesliga-data-shootout/test/019d5b34_1.mp4','/kaggle/working/yolov5')

In [ ]:
os.chdir('/kaggle/working/yolov5')

# YOLOv5 Inference

In [ ]:
!python detect.py --img 1280 \
                  --weights yolov5l6_trained_600images.pt \
                  --source 019d5b34_1.mp4 \
                  --project DFL \
                  --save-txt \
                  --save-conf 

In [ ]:
!cp -r ./DFL/exp/ /kaggle/working/

In [ ]:
!mkdir /kaggle/working/exp/frame

In [ ]:
def video_2_frames(video_file='/kaggle/working/exp/019d5b34_1.mp4', image_dir='/kaggle/working/exp/frame/', image_file='img_%s.png'):
    i = 0
    cap = cv2.VideoCapture(video_file)
    while(cap.isOpened()):
        flag, frame = cap.read()
        if flag == False:
            break
        cv2.imwrite(image_dir+image_file % str(i).zfill(6), frame) 
        i += 1
    cap.release()

In [ ]:
video_2_frames()

In [ ]:
!ls /kaggle/working/exp/frame

In [ ]:
os.chdir('/kaggle/working/')
#!rm -rf yolov5

# Collect label information

In [ ]:
txt_dir='/kaggle/working/exp/labels'
txtpaths=[]
texts=os.listdir(txt_dir)
for item in texts:
    txtpaths+=[os.path.join(txt_dir,item)]

In [ ]:
boxdata=[]
boxfile=[]
for i in range(len(txtpaths)):
    file=txtpaths[i]
    boxdata+=[np.loadtxt(file)]     
    boxfile+=[file[0:-4].split('/')[-1]]
print(boxdata[0:6])
print(boxfile[0:6])

BOX=pd.DataFrame()

for i in range(len(boxdata)):
    if type(boxdata[i][0])==np.float64:
        add=pd.DataFrame([boxdata[i]])
        add[6]=boxfile[i]
        BOX=pd.concat([BOX,add])
    else:
        add=pd.DataFrame(boxdata[i])
        add[6]=boxfile[i]
        #display(add)
        BOX=pd.concat([BOX,add])       

BOX2=BOX.reset_index(drop=True)
BOX2[0:10]

In [ ]:
BOX2.columns=['label','x','y','w','h','confidence','file']
BOX2['order']=BOX2['file'].apply(lambda x:('0000'+x.split('_')[-1])[-4:])
BOX2['time']=BOX2['order'].apply(lambda x:int(x)*30/750)
data0=BOX2.sort_values('order',ascending=True)
display(data0)

In [ ]:
#remove noise(detection errors)
data0=data0[data0['confidence']>0.65]
data0=data0.reset_index(drop=True)
display(data0)

In [ ]:
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
fig=make_subplots(specs=[[{"secondary_y":False}]])
fig.add_trace(go.Scatter(x=data0['order'],y=data0['x'],name='x(LR)'),secondary_y=False,)
fig.add_trace(go.Scatter(x=data0['order'],y=data0['y'],name='y(UD)'),secondary_y=False,)
fig.update_layout(autosize=False,width=700,height=500,title_text="x(LR),y(UD)")
fig.update_xaxes(title_text="order")
fig.update_yaxes(title_text="x(LR),y(UD)",secondary_y=False)
fig.show()

In [ ]:
# trace the ball, watch the movie along with this figure
fig=make_subplots(specs=[[{"secondary_y":False}]])
fig.add_trace(go.Scatter(x=data0.loc[:100,'x'],y=-data0.loc[:100,'y'],name='#1,x(LR),-y(UD)'),secondary_y=False,)
fig.add_trace(go.Scatter(x=data0.loc[100:200,'x'],y=-data0.loc[100:200,'y'],name='#2,x(LR),-y(UD)'),secondary_y=False,)
fig.add_trace(go.Scatter(x=data0.loc[200:300,'x'],y=-data0.loc[200:300,'y'],name='#3,x(LR),-y(UD)'),secondary_y=False,)
fig.add_trace(go.Scatter(x=data0.loc[300:,'x'],y=-data0.loc[300:,'y'],name='#4,x(LR),-y(UD)'),secondary_y=False,)
fig.update_layout(autosize=False,width=700,height=500,title_text="x(LR),-y(UD)")
fig.update_xaxes(title_text="x(LR)")
fig.update_yaxes(title_text="-y(UD)",secondary_y=False)
fig.show()

In [ ]:
image_dir='/kaggle/working/exp/frame/'
def view(order):
    file='img_00'+order+'.png'
    path=os.path.join(image_dir,file)
    img=cv2.imread(path)
    plt.figure(figsize=(12,18))
    plt.imshow(cv2.cvtColor(img,cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
view('0111')# x:Rmax
view('0193')# x:Lmax
view('0257')# y:Dmax
view('0298')# x:Rmax
view('0346')# y:Umax

As compared to pre-trained YOLO weight, weight specialized for ball detection 
was so sensitive that data of confidence level >0.65 are many. And it is possible to trace the ball.
* https://www.kaggle.com/code/stpeteishii/bundesliga-pretrained-yolov5-ball-detection 
* https://www.kaggle.com/code/stpeteishii/bundesliga-pretrained-yolov7-ball-detection 
* https://www.kaggle.com/stpeteishii/bundesliga-yolov5-specialized-for-ball-detection (this notebook)

In [ ]:
nan

In [ ]:
nan